## Imports

In [1]:
import copy
import logging
import os
from pathlib import Path
from typing import Any, Dict, List, Optional

from model_merging.data.dataset import HFImageClassification
from model_merging.model.image_classifier import ImageClassifier
import open_clip
import wandb

import hydra
import omegaconf
import pytorch_lightning as pl
import torch
from hydra import compose, initialize
from hydra.utils import instantiate
from lightning.pytorch import Callback
from omegaconf import DictConfig, ListConfig, OmegaConf
from torch.nn.utils import parameters_to_vector, vector_to_parameters

from nn_core.callbacks import NNTemplateCore
from nn_core.common import PROJECT_ROOT
from nn_core.common.utils import enforce_tags, seed_index_everything
from nn_core.model_logging import NNLogger
from nn_core.serialization import NNCheckpointIO

# Force the execution of __init__.py if this file is executed directly.
import model_merging  # noqa
from model_merging.model.encoder import ClassificationHead, ImageEncoder
from model_merging.model.heads import (
    get_classification_head,
)
from model_merging.utils.io_utils import (
    boilerplate,
    load_model_from_hf,
)
from model_merging.utils.plots import plot_interactive_radar_chart
from model_merging.utils.utils import (
    build_callbacks,
    get_finetuning_accuracies,
    compute_avg_accuracy,
    print_memory,
)
import json
import os

/media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[10:23:21] INFO     PyTorch version 2.8.0 available.                                                   ]8;id=351786;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/datasets/config.py\config.py]8;;\:]8;id=293626;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/datasets/config.py#54\54]8;;\

/media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/lightning_utilities/core/imports.py:14: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/data/datamodule.py:206: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=str(PROJECT_ROOT / "con

In [6]:
import hydra
from hydra import initialize, compose
from typing import Dict, List

hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(version_base=None, config_path=str("../conf"), job_name="layer_analysis")
cfg = compose(config_name="multitask", overrides=["benchmark=hard"])

/media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/IPython/core/async_helpers.py:129: UserWarning: 
'hydra/launcher/basic' is validated against ConfigStore schema with the same name.
This behavior is deprecated in Hydra 1.1 and will be removed in Hydra 1.2.
See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/automatic_schema_matching for migration instructions.
  coro.send(None)


## Boilerplate

In [8]:
pylogger = logging.getLogger(__name__)

In [7]:
seed_index_everything(cfg)

logger, template_core = boilerplate(cfg)

num_tasks = len(cfg.benchmark.datasets)

# Temporarily disable struct mode to allow dynamic update
omegaconf.OmegaConf.set_struct(cfg, False)
cfg.num_tasks = num_tasks  # Now we can safely update it
omegaconf.OmegaConf.set_struct(cfg, True)  # Re-enable struct mode

# upperbound accuracies, used for logging the normalized accuracy
finetuned_accuracies: Dict[str, float] = get_finetuning_accuracies(
    cfg.misc.finetuned_accuracy_path
)[cfg.nn.encoder.model_name]

[10:28:14] INFO     Global seed set to 1608637542                                                        ]8;id=224430;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/lightning_fabric/utilities/seed.py\seed.py]8;;\:]8;id=170108;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/lightning_fabric/utilities/seed.py#53\53]8;;\

           INFO     Setting seed 1608637542 from seeds[0]                                              ]8;id=323618;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/nn_core/common/utils.py\utils.py]8;;\:]8;id=918782;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/nn_core/common/utils.py#107\107]8;;\

           INFO     Tags: ['dev']                                                                       ]8;id=30561;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/nn_core/common/utils.py\utils.py]8;;\:]8;id=398255;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/nn_core/common/utils.py#96\96]8;;\

           INFO     Instantiating <WandbLogger>                                                 ]8;id=852900;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/nn_core/model_logging.py\model_logging.py]8;;\:]8;id=721902;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/nn_core/model_logging.py#41\41]8;;\

/media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(


           INFO     Uploading source code to W&B                                                ]8;id=130475;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/nn_core/model_logging.py\model_logging.py]8;;\:]8;id=464412;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/nn_core/model_logging.py#63\63]8;;\

## Load models

In [11]:
# only has vision encoder, no text transformer
base_model: ImageEncoder = load_model_from_hf(
    model_name=cfg.nn.encoder.model_name
)

finetuned_models = {
    dataset: load_model_from_hf(
        model_name=cfg.nn.encoder.model_name, dataset_name=dataset.name
    ).state_dict()
    for dataset in cfg.benchmark.datasets
}

pylogger.info(f"Number of tasks: {cfg.num_tasks}")
pylogger.info(f"Finetuned models: {list(finetuned_models.keys())}")


[10:38:00] INFO     Loading ViT-B-32 pre-trained weights.                                             ]8;id=480906;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/model/encoder.py\encoder.py]8;;\:]8;id=121525;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/model/encoder.py#21\21]8;;\

           INFO     Loading pretrained ViT-B-32 from OpenAI.                                          ]8;id=575048;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/open_clip/factory.py\factory.py]8;;\:]8;id=810921;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/open_clip/factory.py#82\82]8;;\

[10:38:01] INFO     Removing text transformer from the model.                                         ]8;id=330076;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/model/encoder.py\encoder.py]8;;\:]8;id=815002;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/model/encoder.py#35\35]8;;\

[10:38:02] INFO     Loading ViT-B-32 pre-trained weights.                                             ]8;id=806390;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/model/encoder.py\encoder.py]8;;\:]8;id=383296;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/model/encoder.py#21\21]8;;\

           INFO     Loading pretrained ViT-B-32 from OpenAI.                                          ]8;id=261810;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/open_clip/factory.py\factory.py]8;;\:]8;id=460469;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/open_clip/factory.py#82\82]8;;\

...

[10:38:05] INFO     Loading ViT-B-32 pre-trained weights.                                             ]8;id=888498;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/model/encoder.py\encoder.py]8;;\:]8;id=171544;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/model/encoder.py#21\21]8;;\

           INFO     Loading pretrained ViT-B-32 from OpenAI.                                          ]8;id=277983;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/open_clip/factory.py\factory.py]8;;\:]8;id=912342;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/open_clip/factory.py#82\82]8;;\

[10:38:06] INFO     Removing text transformer from the model.                                         ]8;id=456742;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/model/encoder.py\encoder.py]8;;\:]8;id=887125;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/model/encoder.py#35\35]8;;\

           INFO     Number of tasks: 3                                                             ]8;id=920415;file:///tmp/ipykernel_4020056/2486514914.py\2486514914.py]8;;\:]8;id=726824;file:///tmp/ipykernel_4020056/2486514914.py#13\13]8;;\

           INFO     Finetuned models: [{'_target_': 'model_merging.data.dataset.load_dataset',     ]8;id=6460;file:///tmp/ipykernel_4020056/2486514914.py\2486514914.py]8;;\:]8;id=24388;file:///tmp/ipykernel_4020056/2486514914.py#14\14]8;;\
                    'name': 'Cars', 'preprocess_fn': '???', 'ft_epochs': 35, 'hf_dataset':                         
                    {'_target_': 'datasets.load_dataset', 'path': 'tanganke/stanford_cars'}},                      
                    {'_target_': 'model_merging.data.dataset.load_dataset', 'name': 'KMNIST',                      
                    'ft_epochs': 5, 'preprocess_fn': '???', 'hf_dataset': {'_target_':                             
                    'datasets.load_dataset', 'path': 'tanganke/kmnist'}}, {'_target_':                             
                    'model_merging.data.dataset.load_dataset', 'name': 'RESISC45', 'ft_epochs':                    
                    15, 'preprocess_fn': '???', 'hf_dataset': {'_target_':                                         
                    'datasets.load_dataset', 'path': 'tanganke/resisc45'}, 'label_map': {0: 0, 1:                  
                    2, 2: 3, 3: 5, 4: 6, 5: 7, 6: 9, 7: 10, 8: 11, 9: 12, 10: 13, 11: 14, 12: 15,                  
                    13: 1, 14: 16, 15: 17, 16: 18, 17: 19, 18: 4, 19: 20, 20: 21, 21: 22, 22: 23,                  
                    23: 24, 24: 25, 25: 26, 26: 27, 27: 28, 28: 29, 29: 30, 30: 31, 31: 32, 32: 8,                 
                    33: 33, 34: 34, 35: 35, 36: 36, 37: 37, 38: 38, 39: 39, 40: 40, 41: 41, 42:                    
                    42, 43: 43, 44: 44}, 'classnames_override': ['airplane', 'forest', 'airport',                  
                    'baseball diamond', 'industrial area', 'basketball court', 'beach', 'bridge',                  
                    'river', 'chaparral', 'church', 'circular farmland', 'cloud', 'commercial                      
                    area', 'dense residential', 'desert', 'freeway', 'golf course', 'ground track                  
                    field', 'harbor', 'intersection', 'island', 'lake', 'meadow', 'medium                          
                    residential', 'mobile home park', 'mountain', 'overpass', 'palace', 'parking                   
                    lot', 'railway', 'railway station', 'rectangular farmland', 'roundabout',                      
                    'runway', 'sea ice', 'ship', 'snowberg', 'sparse residential', 'stadium',                      
                    'storage tank', 'tennis court', 'terrace', 'thermal power station',                            
                    'wetland']}]                                                                                   

## Merging

In [12]:
non_matrix_params_aggregation = 'mean'
compression_ratio = 1 / len(cfg.benchmark.datasets)

In [13]:
from model_merging.merging.structured import aggregate_decomposed_task_vectors, decompose_task_vectors, get_svd_dict
from model_merging.utils.utils import apply_dict_to_model, compute_task_dict

task_dicts = {}

for dataset in cfg.benchmark.datasets:
    task_dicts[dataset] = compute_task_dict(
        base_model.state_dict(), finetuned_models[dataset]
    )
    del finetuned_models[dataset]  # Delete one model at a time
    torch.cuda.empty_cache()

print_memory("after computing task dicts")

[10:44:46] WARNING  after computing task dicts -- memory in MB: 4819.03515625                           ]8;id=150796;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/utils/utils.py\utils.py]8;;\:]8;id=245074;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/utils/utils.py#23\23]8;;\

In [14]:
decomposed_task_vectors = decompose_task_vectors(task_dicts, compression_ratio)

multi_task_vector = aggregate_decomposed_task_vectors(
    ref_state_dict=copy.deepcopy(base_model.state_dict()),
    decomposed_task_vectors=decomposed_task_vectors,
    non_matrix_params_aggregation=non_matrix_params_aggregation
)

Summing SVD: 100%|██████████| 158/158 [00:06<00:00, 23.47it/s]


In [15]:
merged_encoder: ImageEncoder = copy.deepcopy(base_model)

merged_encoder = apply_dict_to_model(
    multi_task_vector,
    merged_encoder,
)

In [ ]:
results = {}
print_memory("before eval")
for dataset_cfg in cfg.benchmark.datasets:

    dataset = instantiate(
        dataset_cfg, preprocess_fn=base_model.val_preprocess
    )

    classification_head = get_classification_head(
        cfg.nn.encoder.model_name,
        dataset_cfg.name,
        ckpt_path=cfg.misc.ckpt_path,
        openclip_cachedir=cfg.misc.openclip_cachedir,
        device=cfg.device,
    )

    model = ImageClassifier(
        encoder=merged_encoder,
        classifier=classification_head,
        x_key=cfg.conventions.x_key,
        y_key=cfg.conventions.y_key,
    )

    model.set_metrics(len(dataset.classnames))
    model.set_task(dataset_cfg.name)
    model.set_finetuning_accuracy(
        finetuned_accuracies[
            dataset_cfg.name + "Val" if cfg.eval_on_train else dataset_cfg.name
        ]
    )

    callbacks: List[Callback] = build_callbacks(cfg.train.callbacks, template_core)

    trainer = pl.Trainer(
        default_root_dir=cfg.core.storage_dir,
        plugins=[NNCheckpointIO(jailing_dir=logger.run_dir)],
        logger=logger,
        callbacks=callbacks,
        limit_test_batches=(
            cfg.number_of_train_batches if cfg.eval_on_train else None
        ),
        **cfg.train.trainer,
    )

    if cfg.eval_on_train:
        pylogger.error("For now evaluation supported only on val-set")
        pylogger.info(f"Evaluating on {dataset_cfg.name} the training set")
        test_results = trainer.test(model=model, dataloaders=dataset.train_loader)

    else:
        pylogger.info(f"Evaluating on the {dataset_cfg.name} test set!")
        test_results = trainer.test(model=model, dataloaders=dataset.test_loader)

    results[dataset_cfg.name] = test_results

avg = compute_avg_accuracy(results)
results["avg"] = [
    avg
]  # as a list for consistency due to lightning logging stuff this way

logger.experiment.log(avg)

pylogger.info(results)